In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re
from tqdm.auto import tqdm
import urllib.request
from selectolax.parser import HTMLParser

In [2]:
df = pd.read_pickle('kickstarter.pkl')

In [3]:
sample = df.loc[:200]

Testing which script runs faster for the full dataset.
There are a few parts to change/improve

- How we get the HTML 
- How we parse the HTML 
- How we clean the HTML

# Cleaning the HTML
At the end of it, we know we want to have only the p tags and if possible h1+ tags and ul tags. We can do this by reading each and appending to a list and joining on that list to create a string for the description. 

According to https://waymoot.org/home/python_string/, a list comprehension is the most effective method to joining a list of strings. Thus we'll be implementing this into our pull function for all tests. 

# Acquiring the HTML
Before we process and parse HTML, we have to get the HTML first. The two ways I'm focusing on are utilizing the requests library and the urllib library, r and u for function respectively. 

In [4]:
## Requests & BeautifulSoup
def r_bs_pull(url):
    try:
        result = requests.get(url)
        soup = BeautifulSoup(result.content)
        
        textfield = '\n'.join([item.text for item in soup.find('div', 'full-description').find_all('p')])

        return(textfield)
    except AttributeError:
        return 'Missing Description'
    
## URLLIB & BeautifulSoup
def u_bs_pull(url):
    try:
        result = urllib.request.urlopen(url)
        soup = BeautifulSoup(result.read())
    
        textfield = '\n'.join([item.text for item in soup.find('div', 'full-description').find_all('p')])
    
        return(textfield)
    except AttributeError:
        return 'Missing Description'

In [11]:
%%time
sample['r_bs_pull'] = sample['web_url'].apply(r_bs_pull)
#7 Min 54s for 201 files

CPU times: user 5min 25s, sys: 2.68 s, total: 5min 27s
Wall time: 7min 54s


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
%%time
sample['u_bs_pull'] = sample['web_url'].apply(u_bs_pull)
#3min 17s for 201 files
#3m 32s for 201 files

CPU times: user 29.1 s, sys: 712 ms, total: 29.8 s
Wall time: 3min 32s


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


From this experiment, it looks like using urllib in this case is much faster. 

# Parsing HTML
We've optimized the HTML content so now we need to optimize parsing through the HTML. This comparison will be between BeautifulSoup and selectolax.

In [5]:
# def u_bs_pull(url):
#     try:
#         result = urllib.request.urlopen(url)
#         soup = BeautifulSoup(result.read())
    
#         textfield = '\n'.join([item.text for item in soup.find('div', 'full-description').find_all('p')])
    
#         return(textfield)
#     except AttributeError:
#         return 'Missing Description'
    
def u_sl_pull(url):
    try:
        result = urllib.request.urlopen(url)
        parse = HTMLParser(result.read())
        
        textfield = '\n'.join([node.text() for node in parse.css_first("div.full-description".css('p'))])
        
        return(textfield)
    except AttributeError:
        return 'Missing Description'
    
## Since we've seen u_bs run, we'll just comapre it with u_sl

In [40]:
%%time
sample['u_sl_pull'] = sample['web_url'].apply(u_sl_pull)
# 2min 59s
# 2min 59s

CPU times: user 4.23 s, sys: 922 ms, total: 5.15 s
Wall time: 2min 59s


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


The improvement is very noticable, saving .2 seconds per iteration on average.

I want to apply the p tags and also the ul tag so we'll work with BS and see how fast this'll work. 

In [6]:
def u_bs_pull_plus(url):
    try:
        result = urllib.request.urlopen(url)
        soup = BeautifulSoup(result.read())
    
        textfield = '\n'.join([item.text for item in soup.find('div', 'full-description').find_all(['p','ul'])])
    
        return(textfield)
    except AttributeError:
        return 'Missing Description'

In [42]:
%%time
sample['u_bs_pull_plus'] = sample['web_url'].apply(u_bs_pull_plus)
# 3min 15s.

CPU times: user 29.1 s, sys: 677 ms, total: 29.8 s
Wall time: 3min 15s


/Users/Matt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Since adding the 'ul' tag isn't much different in BS, I'll be using urllibrequest and the beautifulsoup library together. 

In [7]:
from multiprocessing import  Pool
from functools import partial
import numpy as np

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [8]:
parallelize_on_rows(sample, u_bs_pull_plus) 

0      Missing Description
1      Missing Description
2      Missing Description
3      Missing Description
4      Missing Description
5      Missing Description
6      Missing Description
7      Missing Description
8      Missing Description
9      Missing Description
10     Missing Description
11     Missing Description
12     Missing Description
13     Missing Description
14     Missing Description
15     Missing Description
16     Missing Description
17     Missing Description
18     Missing Description
19     Missing Description
20     Missing Description
21     Missing Description
22     Missing Description
23     Missing Description
24     Missing Description
25     Missing Description
26     Missing Description
27     Missing Description
28     Missing Description
29     Missing Description
              ...         
171    Missing Description
172    Missing Description
173    Missing Description
174    Missing Description
175    Missing Description
176    Missing Description
1